# North Atlantic O2 projection
    - 3D isopycnal version
    - This script takes the output of the o2mod_example_CV_v2.ipynb
    - O2 is estimated as a function of T, S, long, lat, year, month (below 100m, month is always March = 2)
    - year is measured since 1965-01
    - Estimation methods include Neural Network and Random Forest Regression
    - Specify the ML you want to use, and the range of years to calculate the O2 field

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sklearn as skl
import gsw
import joblib
import os
from multiprocessing import Pool

In [2]:
# load the ML model & select the basin
model = 'IPSL-CM6A-LR'
Basin = 1
#filename = f'RF_model_{model}.sav'
MLname = 'SNN'
filename = f'sNN_model_{model}.sav'
#filename = f'dNN_model_{model}.sav'
#
! mkdir -p temp
! mkdir -p output

In [3]:
MLmodel = joblib.load(filename)

/glade/work/ito/miniconda3/envs/calc2/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/glade/work/ito/miniconda3/envs/calc2/lib/python3.8/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
# Load the mean and standard deviation values
dataref = np.load(f'ML_params_{model}.npz')#, Xm=Xm, Xstd=Xstd, ym=ym, ystd=ystd)
Xm=dataref['Xm']
Xstd=dataref['Xstd']
ym=dataref['ym']
ystd=dataref['ystd']

In [5]:
# set up the input data for projection
# Define the directory where the CMIP6 data for the model is stored
dir = '/glade/campaign/univ/ugit0034/cmip6/' + model + '/'

# Open the dataset for oxygen concentration for the specified model and time range
ds0 = xr.open_dataset(dir + 'o2_' + model + '_196501-201412.nc')

# Open the dataset for sea surface salinity for the specified model and time range
ds1 = xr.open_dataset(dir + 'so_' + model + '_196501-201412.nc')

# Open the dataset for sea surface temperature for the specified model and time range
ds2 = xr.open_dataset(dir + 'thetao_' + model + '_196501-201412.nc')

In [6]:
# Open the dataset for the basin mask
dsm = xr.open_dataset('/glade/campaign/univ/ugit0034/cmip6/basin_mask_01.nc')

In [7]:
# Get longitude values from the ds0 dataset and convert to a NumPy array
x = ds0.lon.to_numpy()

# Get latitude values from the ds0 dataset and convert to a NumPy array
y = ds0.lat.to_numpy()

# Create a 1D array of longitude values ranging from 0 to 359 with a step size of 1
xi = np.arange(0, 360, 1)

# Create a 1D array of latitude values ranging from 0 to 179 with a step size of 1
yi = np.arange(0, 180, 1)

# Create a 2D grid of longitude and latitude values using meshgrid
xx, yy = np.meshgrid(x, y)

# Create a 2D grid of longitude and latitude values using meshgrid for the new coordinates
xxi, yyi = np.meshgrid(xi, yi)

In [8]:
# Extract the basin mask from the dsm dataset
mask = dsm.basin_mask

# Extract the depth values for oxygen concentration from the o2 dataset and convert to NumPy array
zmod = ds0.depth.to_numpy()

# Extract the depth values for the basin mask from the mask dataset and convert to NumPy array
zmask = mask.depth.to_numpy()

# select a basin mask at the model depth levels
ma = dsm.basin_mask.sel(depth=zmod,method='nearest').to_numpy()

In [9]:
# apply basin mask 
def apply_basinmask(datain,basin):
    #dataout=np.where((ma==basin)&(yy>0),datain,np.nan)
    dataout=np.where((ma==basin)&(yy>0),datain,np.nan)
    return dataout

In [10]:
def moving_average(data,w):
    weight=np.ones(w)/w
    smooth_data=np.convolve(data, weight, mode='same')
    return smooth_data

In [11]:
# get input data from full model
def get_inputdata(zlev,it):
    # Open the dataset for sea surface salinity for the specified model and time range
    ds1 = xr.open_dataset(dir + 'so_' + model + '_196501-201412.nc')
    # Open the dataset for sea surface temperature for the specified model and time range
    ds2 = xr.open_dataset(dir + 'thetao_' + model + '_196501-201412.nc')
    #
    soa=ds1.so.isel(time=it).interp(depth=zlev).to_numpy().squeeze()
    toa=ds2.thetao.isel(time=it).interp(depth=zlev).to_numpy().squeeze()
    return soa,toa

In [12]:
# generate data matrix
def gen_datamatrix(xi,yi,x1,x2,it,xind,yind):
    X1 = x1.flatten() # 
    X2 = x2.flatten() #
    mo = it % 12
    yr = (it - mo) / 12
    X5 = np.ones(np.shape(x1))*yr
    X6 = np.ones(np.shape(x1))*mo
    X5 = X5.flatten()
    X6 = X6.flatten()
    X3 = xind.flatten() # lon
    X4 = yind.flatten() # lat
    xind0 = xind.flatten()
    yind0 = yind.flatten()
    # remove nan
    dd = X1+X2
    X11=X1[np.isnan(dd)==False]
    X21=X2[np.isnan(dd)==False]
    X31=X3[np.isnan(dd)==False]
    X41=X4[np.isnan(dd)==False]
    X51=X5[np.isnan(dd)==False]
    X61=X6[np.isnan(dd)==False]
    Xind = xind0[np.isnan(dd)==False]
    Yind = yind0[np.isnan(dd)==False]
    # Normalize data
    # Create a 2D array 'X' containing dsa1, dta1, xx1, yy1, yr1, mn1 as its rows
    # generate data matrix and standardize it
    X = np.array([X11, X21, X31, X41, X51, X61])
    Xa = (X.T - Xm)/Xstd
    Nsample = np.size(X11)
    #print(Nsample)
    return Xa,Xind,Yind

In [13]:
def map_yearly(year):
    Nx=np.size(x)
    Ny=np.size(y)
    zlev_arr=np.array([zlev])
    o2est2=np.zeros((12,1,Ny,Nx))
    xind,yind=np.meshgrid(np.arange(0,Nx,1),np.arange(0,Ny,1))
    if year%10 == 5:
        print('year = '+str(year))
    t=np.arange(str(year)+'-01',str(year+1)+'-01',dtype='datetime64[M]')
    for month in range(12):
        it = month+(year-1965)*12
        soa,toa = get_inputdata(zlev,it)
        # apply mask
        soa=apply_basinmask(soa,Basin)
        toa=apply_basinmask(toa,Basin)
        # generate data matrix
        Xa,xi,yi=gen_datamatrix(soa,toa,xx,yy,it,xind,yind)
        temp = np.shape(Xa)
        Nsample=temp[0]
        # projection
        out = reg.predict(Xa)
        # map it back to lon-lat grid
        temp = np.nan*np.zeros((Ny,Nx))
        for n in range(Nsample):
            temp[yi[n],xi[n]]=out[n]
        o2est2[month,0,:,:] = temp*ystd + ym
    da1=xr.DataArray(data=o2est2,name='o2est',dims=['time','depth','lat','lon'],
                 coords={'time':t,'depth':zlev_arr,'lat':y,'lon':x})
    ds=da1.to_dataset()
    ds.to_netcdf('temp/o2est_'+str(year)+'.nc')
    return 0

In [14]:
zlevels = zmod.copy()
yrs=np.arange(1965,2015,1)
#
# reconstruction in parallel mode
#
reg=MLmodel
x = ds0.lon.to_numpy()
y = ds0.lat.to_numpy()
#
for zlev_cnt,zlev in enumerate(zlevels):
    print(f'calculating {zlev}m')
    maz = dsm.basin_mask.interp(depth=zlev).to_numpy()
    #kind=[idx for idx,elem in enumerate(Z) if elem==zlev]
    #maz=np.squeeze(ma[kind,:,:])
    os.system('rm temp/*.nc')
    #
    if __name__ == '__main__':
        with Pool(10) as p:
            print(p.map(map_yearly, yrs))
    #
    # save the result as a netCDF file
    #
    dtemp=xr.open_mfdataset('temp/o2est*.nc')
    dtemp.to_netcdf(f'output/O2_'+model+'_z'+str(int(zlev))+'.nc')

calculating 0.0m
year = 1965year = 1975

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 10.0m
year = 1975year = 1965

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 20.0m
year = 1975year = 1965

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 30.0m
year = 1975year = 1965

year = 1985
year = 1995
year = 2005
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calculating 50.0m
year = 1975year = 1965

year = 1985
year = 1995
year = 2005
[0, 0, 

In [15]:
dtemp.to_netcdf(f'output/O2_'+model+'_z'+str(int(zlev))+'.nc')

In [16]:
ds=xr.open_mfdataset(f'output/O2_{model}_z*')
ds.to_netcdf(f'output/O2_{model}_{MLname}_monthly.nc')